# Prompt Examples

Here, we'll be extracting phone numbers from text. We'll do it with an without examples to illustrate the idea. 

In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.insert(0, "../../")

In [2]:
from kor.extraction import Extractor
from kor.nodes import Object, Text, Number
from langchain.chat_models import ChatOpenAI

In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", 
    temperature = 0,
    max_tokens = 2000,
    frequency_penalty = 0,
    presence_penalty = 0,
    top_p = 1.0,
)
model = Extractor(llm)

## No Examples

Sometimes it's sufficient to specify what to extract without providing any examples.

In [4]:
schema = Text(
    id="phone_number",
    description=(
        "Any phone numbers found in the text. Should be output in the format:"
        " (xxx)-xxx-xxxx"
    ),
)

In [5]:
model(
    (
        "My phone number is (123)-444-9999. I found my true love on a blue sunday."
        " Her number was (333)1234321. I had no phone when yesterday. "
        "I received a call from 8881111222"
    ),
    schema,
)

{'phone_number': ['(123)-444-9999', '(333)-123-4321', '888-111-1222']}

And the prompt that was used is:

In [6]:
print(model.prompt_generator.format_as_string("[user input goes here]", schema))

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

{
 phone_number: string[] // Any phone numbers found in the text. Should be output in the format: (xxx)-xxx-xxxx
}
```


For Union types the output must EXACTLY match one of the members of the Union type.

Please enclose the extracted information in HTML style tags with the tag name corresponding to the corresponding component ID. Use angle style brackets for the tags ('>' and '<'). Only output tags when you're confident about the information that was extracted from the user's query. If you can extract several pieces of relevant information from the query, then include all of them. If the type is an array, please repeat the corresponding tag name multiple times once for each relevant extraction. Do NOT output

## Add One Example

Let's add one example to see whether it helps format the last phone number correctly.

In [7]:
schema = Text(
    id="phone_number",
    description=(
        "Any phone numbers found in the text. Should be output in the format:"
        " (xxx)-xxx-xxxx"
    ),
    examples=[("Someone called me from 1231231234", "(123)-123-1234")],
)

model(
    (
        "My phone number is (123)-444-9999. I found my true love on a blue sunday."
        " Her number was (333)1234321. I had no phone when yesterday. "
        "I received a call from 8881111222"
    ),
    schema,
)

{'phone_number': ['(123)-444-9999', '(333)-123-4321', '(888)-111-1222']}

And the prompt that was used is:

In [8]:
print(model.prompt_generator.format_as_string("[user input goes here]", schema))

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

{
 phone_number: string[] // Any phone numbers found in the text. Should be output in the format: (xxx)-xxx-xxxx
}
```


For Union types the output must EXACTLY match one of the members of the Union type.

Please enclose the extracted information in HTML style tags with the tag name corresponding to the corresponding component ID. Use angle style brackets for the tags ('>' and '<'). Only output tags when you're confident about the information that was extracted from the user's query. If you can extract several pieces of relevant information from the query, then include all of them. If the type is an array, please repeat the corresponding tag name multiple times once for each relevant extraction. Do NOT output